## Eventuelle skjevheter
Selv om datasettet ble vurdert som komplett og innenfor realistiske verdier i databehandlingsfasen, er det viktig å reflektere rundt mulige skjevheter og hvordan de kunne påvirke analysen. Ettersom vi arbeider med tidsseriedata for havnivå, kan skjevheter oppstå i form av feilmålinger/uvanlige hopp eller trendforskyvning over tid. Trendforsyvninger kan lett identifiseres ved bruk av et linjediagram i visualiseringsdelen. 

Hadde vi oppdaget ekstreme verdier som lå langt utenfor realistiske havnivå i databehandlingsfasen, ville vi håndtert dette ved bruk av sett_usannsynlige_til_nan() funksjonen og bruke rens_manglende_verdier() videre i analysen og visualiseringen for å få komplette grafer. Da vil NaN eller outliners ble fylt inn ved bruk av valgt metode som f.eks. interpolasjon eller ffill. 

## Bruk av describe() for å undersøke datasettet
Denne funksjonen gir et overordnet bildet av dataen og kan gjøre brukeren oppmerksom på ulike trender eller øyeåpnere. Tabellen viser at standardavviket for mean er svært lavt mens hos både min og max-verdiene er standardavviket mye høyere, noe som indikerer stor variasjon i de ekstreme havnivåmålingene. Dette er forventet fordi havnivået vil variere betydelig fra måned til måned som følge av faktorer som geografisk området, tidevann, stormer, vindforhold og regionale havstrømmer. Ved å undersøke verdiene i de ulike kvartilene og medianen ser vi at min verdiene synker over tid og max og mean-verdiene stiger over tid.

In [3]:
from pathlib import Path
import sys

project_root = Path.cwd().parents[1]
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Dataanalyse import beskriv_statistikk

beskriv_statistikk("../../data/Havnivådata/havnivaadata.json")

,min,mean,max
count,386.000000,386.000000,386.000000
mean,-0.366911,0.029373,0.434296
std,0.104515,0.028931,0.120165
min,-0.727860,-0.022326,0.212865
25%,-0.417572,0.005810,0.348452
50%,-0.351001,0.024730,0.409959
75%,-0.289406,0.053100,0.483461
max,-0.188627,0.093230,0.837300


## Beregning av gjennomsnitt, median og standardavvik ved bruk av NumPy
Benytter NumPy til å beregne statistiske mål for kolonnene min, mean og max i datasettet. Ved bruk av funksjonene np.mean(), np.median() og np.std() får vi verdier som beskriver sentral verdi og variasjon i dataene. Dette viser hvordan NumPy kan brukes som et selvstendig verktøy for statistisk analyse, og gir bedre forståelse av hvordan disse målene faktisk beregnes.

In [1]:
from pathlib import Path
import sys

project_root = Path.cwd().parents[1]
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Dataanalyse import beregn_statistikk_numpy

statistikk = beregn_statistikk_numpy("../../data/Havnivådata/havnivaadata.json")
for kol, stats in statistikk.items():
    print(f"\n  Statistikk for '{kol}':")
    for navn, verdi in stats.items():
        print(f"  {navn.capitalize()}: {verdi:.5f}")



  Statistikk for 'min':
  Gjennomsnitt: -0.36691
  Median: -0.35100
  Standardavvik: 0.10452

  Statistikk for 'mean':
  Gjennomsnitt: 0.02937
  Median: 0.02473
  Standardavvik: 0.02893

  Statistikk for 'max':
  Gjennomsnitt: 0.43430
  Median: 0.40996
  Standardavvik: 0.12017


### Undersøker eventuell korrelasjon mellom gjennomsnittlig havnivåstigning og tid
For å undersøke hvorvidt havnivået har steget jevnt over tid, beregnes Pearson-korrelasjon mellom måned og gjennomsnittlig havnivå. Verdien varierer mellom -1 og 1. En korrelasjon nær 1 indikerer en sterk positiv sammenheng (havnivået stiger over tid). En korrelasjon nær -1 indikerer en sterk negativ sammenheng (havnivået synker). En korrelasjon nær 0 betyr at det ikke er noen tydelig sammenheng.

In [2]:
from pathlib import Path
import sys

project_root = Path.cwd().parents[1]
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Dataanalyse import beregn_korrelasjon_mnd_vs_mean

beregn_korrelasjon_mnd_vs_mean("../../data/Havnivådata/havnivaadata.json")



 Pearson korrelasjon mellom måneder og havnivåstigning: 0.976
 Sterk positiv sammenheng! – havnivået stiger jevnt over tid.


## Vurdering av stabilitet i "mean" kolonnen
Ønsker å identifisert noen uvanlige hopp eller feilverdier i mean-verdiene fra måned til måned, basert på grensen satt til tre ganger standardavvik. Dette er gunstig å vite før man trekker eventuelle konklusjoner.

In [3]:
from pathlib import Path
import sys

project_root = Path.cwd().parents[1]
src_path = project_root / "src" / "Havnivådata"
sys.path.append(str(src_path))

from Dataanalyse import finn_mistenkelige_hopp

avvik = finn_mistenkelige_hopp("../../data/Havnivådata/havnivaadata.json")
display(avvik)
if avvik.empty:
    print("Ingen uvanlige hopp i dataserien ble oppdaget. Datasettet i mean kolonnen vurderes som stabilt.")

,iso_time,mean,mean_diff


Ingen uvanlige hopp i dataserien ble oppdaget. Datasettet i mean kolonnen vurderes som stabilt.
